In [90]:
import torch
from torch import nn
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
from smokingml.datasets.nursing_dataset_v1 import WINSIZE
from smokingml.datasets.nursing_dataset_v1.dataloading import load_one_session, load_one_windowed_session
from smokingml.datasets.nursing_dataset_v1.utils import window_session, train_test_split_windows
from smokingml.modules import train_loop, optimization_loop, evaluate_loop

# 1st LSTM: One Sequence of Points
- entire session is sequence of inputs
- each input is one data point (xyz)
- input shape to model is ($L$,$H_{in}$)

$L =$ `len(session)`

$H_{in} = 3$

In [2]:
data_dir = Path('../data/nursingv1_dataset/')
device = 'cuda:0'

In [3]:
class LSTM1(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=3, hidden_size=64, bias=False)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )

    def forward(self, x):
        # x is len(session) x 3
        o, (h,c) = self.lstm(x) # o is shape (len(session), 64)
        logits = self.mlp(o[-1])

        return logits

model = LSTM1().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [7]:
X,_ = load_one_session(data_dir, 0) 

# logits is the classification for the final point in X
logits = model(X.to(device).float())

# 2nd LSTM: Many sequences of points
- session is padded and windowed into `len(session)` windows
- each window is shaped into sequence of 101 3D inputs (xyz)
- input to LSTM is ($N$, $L$,$H_{in}$)

$N = 64$

$L = 101$

$H_{in} = 3$

In [128]:
class LSTM2(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=3, hidden_size=64, bias=False, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )

    def forward(self, x):
        # x is batch_size x 303, want shape: batch_size x 101 x 3
        x = torch.cat(x.unsqueeze(2).split(WINSIZE, dim=1), axis=2)
        o, (h,c) = self.lstm(x) # o is shape (batch_size, winsize, 64)
        logits = self.mlp(o[:,-1,:])

        return logits

model = LSTM2().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [129]:
dataset = load_one_windowed_session(data_dir, 0)
# X = dataset.tensors[0]
# X = torch.cat(X.unsqueeze(2).split(WINSIZE, dim=1), axis=2)
# dataset = TensorDataset(X, dataset.tensors[1])
loader = DataLoader(dataset, shuffle=True, batch_size=64)

# Not really accurate, since y here are labels for center of windows,
# and this model is predicted the final point in the window
train_loop(model, loader, criterion, optimizer, 10, device)

Starting train_loop


Epoch 9: Train Loss: 0.0050308: 100%|██████████| 10/10 [00:07<00:00,  1.33it/s]

Finished train_loop. Elapsed time: 7.527


# 3rd LSTM: Many Sequences of Windows
- session is padded and windowed into `len(session)` windows
- each window is sequence of 303 inputs (flattened xyz)
- input to model is $(N,L, $H_{in}$)$, $N=64$, $L=303$, $H_{in}=1$
- in model, each window is padded and windowed into 303 subwindows of length 15 (shape 303x15) before theyre fed to lstm
    - in window, first 101 are x, next 101 are y, etc.
    - in subwindow, first 5 are x, next 5 are z, etc.
- input to LSTM is ($N$, $L$, $H_{in}$)

$N = 64$

$L = 303$

$H_{in} = 15$

In [ ]:
class LSTM3(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=15, hidden_size=64, bias=False, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
    
    def window_window(self, x, subwinsize):
        # pad x
        x = torch.pad(x, (0,0,subwinsize//2,subwinsize//2),0,'constant')

        # window x
        

    def forward(self, x):
        # x is batch_size x 303, want shape: batch_size x 303 x 15
        self.window_window(x, subwinsize=5)
        o, (h,c) = self.lstm(x) # o is shape (batch_size, winsize, 64)
        logits = self.mlp(o[:,-1,:])

        return logits

model = LSTM3().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

# 4th LSTM: CNN -> LSTM